<a href="https://colab.research.google.com/github/shabansatti/Global-Controller-for-Transient-Stability-and-Voltage-Regulation/blob/V2---Transient-Controller-through-Ricatti/Voltage_(Global)_Controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install control
from math import *
import numpy as np

import control.matlab as mat
xd_t = 0.657;
xT = 0.127;
xL = 0.24265 * 2;
xds_t = xd_t + xT + xL/2;
angle_min = 10 * 3.14159 / 180 ;
angle_max = 130 * 3.14159 / 180 ;
Vt_min = 0.8;
Vt_max = 1.1;
Pe_min = 0.05;
Pe_max = 1.2;
xs = xT + xL/2;
Vs = 1.3965;
Vt = 1;
R=1;
w0=1;
Td0_t = 6.9;
Q  = np.array ([[500, 0,0], [0, 100, 0], [0, 0, 20]]);


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.1/455.1 kB 9.6 MB/s eta 0:00:00


In [2]:

f1 =  lambda angle, Vt, Pe: -(pow(Pe,2) * pow(xs,2) * cos(angle)) / (
    Vt * pow(Vs,2) * pow(sin(angle),3)) - (xs * xd_t * Pe) / (Vt * xds_t * pow((sin(angle)),2));

f2 = lambda angle, Vt, Pe: (Pe * pow(xs,2)) / (Vt * pow(Vs,2) * pow(sin(angle),2)) + (
    xs * xd_t * cos(angle)) / (Vt * xds_t * (sin(angle)));



f1max = -0.004
f1min = -0.877

f2max = 3.024
f2min = 0.031


#f1max = f1 (angle_max, Vt_max, Pe_min)
#f1min = f1 (angle_min, Vt_min, Pe_max)

#f2max = f2 (angle_min, Vt_min, Pe_max)
#f2min = f2 (angle_max, Vt_min, Pe_min)

f1_mean = 0.5 * (f1max + f1min);
f2_mean = 0.5 * (f2max + f2min);


print ("f1max", f1max)
print ("f1min", f1min)

print ("f2max", f2max)
print ("f2min", f2min)
Av = np.array ([[0, (-5 / (2 * 4)), (-w0/ (2 * 4))], [0, 0, (-1 / 6.9)], [0, f1_mean, (-f2_mean / Td0_t)]]);
Bv = np.array ([[0], [(1 / Td0_t)], [(f2_mean / Td0_t)]]);
D = (np.array([[0], [0], [1]]));
E1 = np.array([[f1max-f1_mean], [0], [0]]);
E2 = np.array([[0, -(f2max-f2_mean)/Td0_t , 0]]);

print ("Bv Shape", np.shape(Bv))
F = (f2max-f2_mean)/Td0_t;
r1 = 1;
lambda1 = 0.5;
lambda2 = 0.5;
epsilon = 2;

A = Av - lambda2*(1/R)*F* Bv @ E2;

B = np.array([sqrt(1/(lambda1+lambda2 * epsilon)) * D, sqrt (lambda2 / epsilon) * Bv * F, sqrt (2 / epsilon) * Bv])
#B = np.array ([[0., 0., 0.], [0., 0.0283712, 0.14492754], [0.81649658, 0.06981732, 0.35664516]])
B = np.array ([[0., 0., 0.], [0., 0.0283712, 0.14492754], [0.81649658, 0.06981732, 0.35664516]])
print (B)
#X  = mat.care(A, B, Q, R=None)[2]
#Kv =np.array ([[0, 10.11, -30.81, -40.14]])

P  = mat.care(A, B, Q, R=None)[0]
L  = mat.care(A, B, Q, R=None)[1]
X = (pow(r1,2)/lambda1) * D.T * P;
print (X)
print ("P: ", P)
print ("X: ", X)
print ("L: ", L)

Kv = Bv.T * P * 1 / epsilon;

print ("Kv: ", Kv);

f1max -0.004
f1min -0.877
f2max 3.024
f2min 0.031
Bv Shape (3, 1)
[[0.         0.         0.        ]
 [0.         0.0283712  0.14492754]
 [0.81649658 0.06981732 0.35664516]]
[[-11.1046241   -9.97248485  10.68198738]]
P:  [[ 535.07464276 -134.6067102    -5.55231205]
 [-134.6067102   120.79316133   -4.98624243]
 [  -5.55231205   -4.98624243    5.34099369]]
X:  [[-11.1046241   -9.97248485  10.68198738]]
L:  [-3.96733921+0.j         -1.30791868+0.64430662j -1.30791868-0.64430662j]
Kv:  [[-10.368686     8.20120841   0.22986416]]


In [3]:
print (f1 (angle_max, Vt_max, Pe_max))
print (f1 (angle_min, Vt_max, Pe_max))
print (f1 (angle_max, Vt_min, Pe_max))
print (f1 (angle_min, Vt_min, Pe_max))
print (f1 (angle_max, Vt_min, Pe_min))
print (f1 (angle_min, Vt_min, Pe_min))
print (f1 (angle_max, Vt_max, Pe_min))
print (f1 (angle_min, Vt_max, Pe_min))


print("f2")
print (f2 (angle_max, Vt_max, Pe_max))
print (f2 (angle_min, Vt_max, Pe_max))
print (f2 (angle_max, Vt_min, Pe_max))
print (f2 (angle_min, Vt_min, Pe_max))
print (f2 (angle_max, Vt_min, Pe_min))
print (f2 (angle_min, Vt_min, Pe_min))
print (f2 (angle_max, Vt_max, Pe_min))
print (f2 (angle_min, Vt_max, Pe_min))



-0.3086060641329128
-25.809079807403137
-0.4243333381827551
-35.48748473517931
-0.024881341829763374
-0.5314940967837162
-0.018095521330736997
-0.3865411612972482
f2
-0.05019799988620624
3.7544417764854985
-0.06902224984353361
5.1623574426675605
-0.24065417126951058
1.8221944455331203
-0.17502121546873495
1.3252323240240875
